## Tests for additive model with three Gaussian inputs
Exact permutation

In [45]:
    var_1 = 1
    var_2 = 1
    var_3 = 1
    
    # rho is the correlation coefficient, and thus, in range [-1,1]. Correlation between X2 and X3 only, i.e. rho = Corr[X2, X3].
    rho = 0.3
    covariance = rho * np.sqrt(var_1) * np.sqrt(var_3)
    
    var_y = 2
    
    np.var()
    
    share = 0.5 * rho ** 2
    
    true_shapley_1 = ((var_1 * (1 - share)) + ((var_2 * var_3) * (1/6) * (rho ** 2)))
    true_shapley_2 = ((1/6) * (var_2 * var_3) * (3 + rho ** 2))
    true_shapley_3 = ((0.5 * (var_1 * rho ** 2)) + ((1/6) * (var_2 * var_3) * (3 - 2 * rho ** 2)))

print(true_shapley_1 + true_shapley_2 + true_shapley_3, var_y)

2.0 2.6


### Difference between the two definitions of the test functions for the additive model
The functions test_get_shapley_additive_three_inputs_alt() and test_get_shapley_additive_three_inputs() differ only with respect to how the variance of the output, $Var[Y]$ is calculated. In the alternate definition $Var[Y]$ is defined as the sum across the Shapley effects, whreas in the latter version $Var[Y]$ is simulated from the model.

In [34]:
def test_get_shapley_additive_three_inputs_alt():
    def additive_model(x):
        return x[:, 0] + x[:, 1] * x[:, 2]

    def x_all(n):
        return cp.MvNormal(mean, cov).sample(n)

    def x_cond(n, subset_j, subsetj_conditional, xjc):
        if subsetj_conditional is None:
            cov_int = np.array(cov).take(subset_j, axis=1)[subset_j]
            distribution = cp.MvNormal(mean[subset_j], cov_int)
            return distribution.sample(n)
        else:
            return _r_condmvn(
                n,
                mean=mean,
                cov=cov,
                dependent_ind=subset_j,
                given_ind=subsetj_conditional,
                x_given=xjc,
            )

    np.random.seed(123)
    n_inputs = 3
    mean = np.zeros(n_inputs)
    var_1 = 1
    var_2 = 1
    var_3 = 1
    
    # rho is the correlation coefficient, and thus, in range [-1,1]. Correlation between X2 and X3 only, i.e. rho = Corr[X2, X3].
    rho = 0.3
    covariance = rho * np.sqrt(var_1) * np.sqrt(var_3)
    share = 0.5 * rho ** 2

    
    cov = np.array(
        [[var_1, 0, covariance],
        [0, var_2, 0],
        [covariance, 0, var_3]]
        )
    
    true_shapley_1 = ((var_1 * (1 - share)) + ((var_2 * var_3) * (1/6) * (rho ** 2)))
    true_shapley_2 = ((1/6) * (var_2 * var_3) * (3 + rho ** 2))
    true_shapley_3 = ((0.5 * (var_1 * rho ** 2)) + ((1/6) * (var_2 * var_3) * (3 - 2 * rho ** 2)))
    
    var_y = (true_shapley_1 + true_shapley_2 + true_shapley_3)
    
    # Normalize Shapley effects by sum of all Shapley effects.
    true_shapley_1 = true_shapley_1/var_y
    true_shapley_2 = true_shapley_1/var_y
    true_shapley_3 = true_shapley_1/var_y
    

    method = "exact"
    n_perms = None
    n_output = 10 ** 4
    n_outer = 10 ** 3
    n_inner = 10 ** 2

    col = ["X" + str(i) for i in np.arange(n_inputs) + 1]
    names = ["Shapley effects", "std. errors", "CI_min", "CI_max"]

    expected = pd.DataFrame(
        data=[
            [true_shapley_1, true_shapley_2, true_shapley_3],
            [0, 0, 0],
            [true_shapley_1, true_shapley_2, true_shapley_3],
            [true_shapley_1, true_shapley_2, true_shapley_3],
        ],
        index=names,
        columns=col,
    ).T

    calculated = get_shapley(
        method,
        additive_model,
        x_all,
        x_cond,
        n_perms,
        n_inputs,
        n_output,
        n_outer,
        n_inner,
    )

    aaae(calculated["Shapley effects"], expected["Shapley effects"], 4)

In [35]:
test_get_shapley_additive_three_inputs_alt()

AssertionError: 
Arrays are not almost equal to 4 decimals

Mismatched elements: 3 / 3 (100%)
Max absolute difference: 0.02068543
Max relative difference: 0.08530075
 x: array([0.4853, 0.2515, 0.2632])
 y: array([0.485 , 0.2425, 0.2425])

In [36]:
def test_get_shapley_additive_three_inputs():
    def additive_model(x):
        return x[:, 0] + x[:, 1] * x[:, 2]

    def x_all(n):
        return cp.MvNormal(mean, cov).sample(n)

    def x_cond(n, subset_j, subsetj_conditional, xjc):
        if subsetj_conditional is None:
            cov_int = np.array(cov).take(subset_j, axis=1)[subset_j]
            distribution = cp.MvNormal(mean[subset_j], cov_int)
            return distribution.sample(n)
        else:
            return _r_condmvn(
                n,
                mean=mean,
                cov=cov,
                dependent_ind=subset_j,
                given_ind=subsetj_conditional,
                x_given=xjc,
            )

    np.random.seed(123)
    n_inputs = 3
    mean = np.zeros(n_inputs)
    var_1 = 1
    var_2 = 1
    var_3 = 1
    
    # rho is the correlation coefficient, and thus, in range [-1,1]. Correlation between X2 and X3 only, i.e. rho = Corr[X2, X3].
    rho = 0.3
    covariance = rho * np.sqrt(var_1) * np.sqrt(var_3)
    share = 0.5 * rho ** 2
    n_sim = 10 ** 6
    
    cov = np.array(
        [[var_1, 0, covariance],
        [0, var_2, 0],
        [covariance, 0, var_3]]
        )
    
    var_y = simulate_variance(
        model=additive_model,
        cov=cov,
        mean=mean,
        n_sim=n_sim)
    
    true_shapley_1 = ((var_1 * (1 - share)) + ((var_2 * var_3) * (1/6) * (rho ** 2)))/var_y
    true_shapley_2 = ((1/6) * (var_2 * var_3) * (3 + rho ** 2))/var_y
    true_shapley_3 = ((0.5 * (var_1 * rho ** 2)) + ((1/6) * (var_2 * var_3) * (3 - 2 * rho ** 2)))/var_y
    
    

    method = "exact"
    n_perms = None
    n_output = 10 ** 4
    n_outer = 10 ** 3
    n_inner = 10 ** 2

    col = ["X" + str(i) for i in np.arange(n_inputs) + 1]
    names = ["Shapley effects", "std. errors", "CI_min", "CI_max"]

    expected = pd.DataFrame(
        data=[
            [true_shapley_1, true_shapley_2, true_shapley_3],
            [0, 0, 0],
            [true_shapley_1, true_shapley_2, true_shapley_3],
            [true_shapley_1, true_shapley_2, true_shapley_3],
        ],
        index=names,
        columns=col,
    ).T

    calculated = get_shapley(
        method,
        additive_model,
        x_all,
        x_cond,
        n_perms,
        n_inputs,
        n_output,
        n_outer,
        n_inner,
    )

    aaae(calculated["Shapley effects"], expected["Shapley effects"], 4)

In [37]:
test_get_shapley_additive_three_inputs()

AssertionError: 
Arrays are not almost equal to 4 decimals

Mismatched elements: 3 / 3 (100%)
Max absolute difference: 0.00969314
Max relative difference: 0.03759508
 x: array([0.4813, 0.2512, 0.2675])
 y: array([0.4856, 0.2578, 0.2578])

In [38]:
first_row = np.zeros(2)
second_row = np.ones(2)
pd.DataFrame(
    data=[
        [first_row],
        [second_row]
    ]
            )

,0
0,"[0.0, 0.0]"
1,"[1.0, 1.0]"
